In [24]:
import argparse
import pandas as pd
from IPython.display import display, HTML

from google.cloud import videointelligence

# Set sample videos. 
path = "gs://vision-ai-experimentation/videos/who.mp4"
#path = "gs://vision-ai-experimentation/videos/botanic_garden_video.mov"

# Processing video.
video_client = videointelligence.VideoIntelligenceServiceClient()
features = [videointelligence.enums.Feature.LABEL_DETECTION]
operation = video_client.annotate_video(path, features=features)
print('\nProcessing video for label annotations:')

result = operation.result(timeout=90)
print('\nFinished processing.')

# Get segment label annotations.
segment_labels = result.annotation_results[0].segment_label_annotations
print("segment_labels: {}".format(segment_labels))

# Set lists for DataFrame.
labels = []
start_times = []
end_times = []
positions = []
confidences = []

for i, segment_label in enumerate(segment_labels):

    print('Video label description: {}'.format(segment_label.entity.description))
    labels.append(segment_label.entity.description)
    
    for category_entity in segment_label.category_entities:
        print('\tLabel category description: {}'.format(
            category_entity.description))

    for i, segment in enumerate(segment_label.segments):
        start_time = (segment.segment.start_time_offset.seconds +
                      segment.segment.start_time_offset.nanos / 1e9)
        end_time = (segment.segment.end_time_offset.seconds +
                    segment.segment.end_time_offset.nanos / 1e9)
        positions = '{}s to {}s'.format(start_time, end_time)
        confidence = segment.confidence
        print('\tSegment {}: {}'.format(i, positions))
        print('\tConfidence: {}'.format(confidence))

        start_times.append(start_time)
        end_times.append(end_time)
        confidences.append(confidence)
    print('\n')

# Display DataFrame as Table.
df = pd.DataFrame({
    "segment_label": labels,
    "start_time": start_times,
    "end_times": end_times,
    "confidence": confidences
})

display(df)
print("Done")


Processing video for label annotations:

Finished processing.
segment_labels: [entity {
  entity_id: "/m/01xr66"
  description: "spokesperson"
  language_code: "en-US"
}
category_entities {
  entity_id: "/m/01g317"
  description: "person"
  language_code: "en-US"
}
segments {
  segment {
    start_time_offset {
    }
    end_time_offset {
      seconds: 188
      nanos: 21166000
    }
  }
  confidence: 0.49944445490837097
}
]
Video label description: spokesperson
	Label category description: person
	Segment 0: 0.0s to 188.021166s
	Confidence: 0.49944445490837097




,segment_label,start_time,end_times,confidence
0,spokesperson,0.0,188.021166,0.499444


Done
